# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from g_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
P1_df = pd.read_csv("C:/Users/19739/Desktop/New folder/API_Challenge_2.0/output_data/_weather.csv")
P1_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pisco,-13.7000,-76.2167,65.01,72,87,11.50,PE,1631208503
1,Lagoa,39.0500,-27.9833,72.50,80,100,7.43,PT,1631208536
2,Tasiilaq,65.6145,-37.6368,48.36,48,4,14.74,GL,1631208536
3,Port-Gentil,-0.7193,8.7815,80.56,78,40,13.71,GA,1631208536
4,Albanel,48.8832,-72.4487,71.74,68,72,12.66,CA,1631208537
...,...,...,...,...,...,...,...,...,...
582,Matagami,49.7502,-77.6328,51.91,93,90,12.66,CA,1631208690
583,Tongliao,43.6125,122.2653,52.63,96,50,2.26,CN,1631208690
584,Calama,-22.4667,-68.9333,70.29,11,20,13.80,CL,1631208475
585,La Ronge,55.1001,-105.2842,64.65,59,75,8.05,CA,1631208690


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = P1_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = P1_df[["Lat", "Lng"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

criteria_city_df = P1_df.loc[(P1_df["Max Temp"] > 70) & (P1_df["Max Temp"] < 80) & (P1_df["Cloudiness"] == 0) & (P1_df["Wind Speed"] <= 10), :]
criteria_city_df = criteria_city_df.dropna(how='any')
criteria_city_df.reset_index(inplace=True)
del criteria_city_df['index']
criteria_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Grasse,43.6667,6.9167,73.26,59,0,4.61,FR,1631208561
1,Temba,-25.3803,28.2631,71.10,28,0,8.99,ZA,1631208593
2,Cape Town,-33.9258,18.4232,72.10,52,0,1.99,ZA,1631208439
3,Wencheng,33.0559,113.7978,71.94,89,0,5.41,CN,1631208601
4,Ürümqi,43.8010,87.6005,70.75,33,0,6.71,CN,1631208607
5,Dimona,31.0667,35.0333,79.84,35,0,5.57,IL,1631208634
6,Villarrica,-25.7500,-56.4333,74.07,30,0,8.03,PY,1631208638
7,Tongchuanshi,35.0806,109.0897,70.52,62,0,2.68,CN,1631208665
8,Tūkrah,32.5341,20.5791,76.78,59,0,7.76,LY,1631208679


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

hotel_df = pd.DataFrame(criteria_city_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df

,City,Country,Lat,Lng
0,Grasse,FR,43.6667,6.9167
1,Temba,ZA,-25.3803,28.2631
2,Cape Town,ZA,-33.9258,18.4232
3,Wencheng,CN,33.0559,113.7978
4,Ürümqi,CN,43.8010,87.6005
5,Dimona,IL,31.0667,35.0333
6,Villarrica,PY,-25.7500,-56.4333
7,Tongchuanshi,CN,35.0806,109.0897
8,Tūkrah,LY,32.5341,20.5791


In [7]:

# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(hotel_df["City"])):

    lat = hotel_df.loc[city]["Lat"]
    lng = hotel_df.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "Hotel",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Grasse,FR,43.6667,6.9167,Grasse
1,Temba,ZA,-25.3803,28.2631,Pretoria Rural
2,Cape Town,ZA,-33.9258,18.4232,Cape Town
3,Wencheng,CN,33.0559,113.7978,Zhumadian
4,Ürümqi,CN,43.8010,87.6005,Ürümqi
5,Dimona,IL,31.0667,35.0333,Dimona
6,Villarrica,PY,-25.7500,-56.4333,Villarrica
7,Tongchuanshi,CN,35.0806,109.0897,Tongchuan
8,Tūkrah,LY,32.5341,20.5791,Tocra


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:

# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))